In [2]:
import simpy

def example(env):
    event = simpy.events.Timeout(env, delay=1, value=42)
    value = yield event
    print('now=%d, value=%d' % (env.now, value))
    
env = simpy.Environment()
p = env.process(example(env))

env.run()

now=1, value=42


In [15]:
def example(env):
    yield env.timeout(1)
    env.exit(42) 

env = simpy.Environment()
p = env.process(example(env))
env.run(until=p)

42

In [9]:
def my_proc(env):
    yield env.timeout(1)
    print(env.now)
    return 'Monty Python’s Flying Circus'
env = simpy.Environment()
proc = env.process(my_proc(env))

env.run(until=proc)

1


'Monty Python’s Flying Circus'

In [5]:
def subfunc(env):
    print(env.active_process)  # will print "p1"

def my_proc(env):
    while True:
        print(env.active_process)  # will print "p1"
        subfunc(env)
        yield env.timeout(1)

env = simpy.Environment()
p1 = env.process(my_proc(env))
env.active_process  # None
env.step()
env.active_process  # None

<Process(my_proc) object at 0x110643748>
<Process(my_proc) object at 0x110643748>


In [16]:
def my_callback(event):
    print('Called back from',event)
    
env = simpy.Environment()
event = env.event()
event.callbacks.append(my_callback)
event.callbacks

[<function __main__.my_callback>]

In [19]:
class School:
    def __init__(self, env):
        self.env = env
        self.class_ends = env.event()
        self.pupil_procs = [env.process(self.pupil()) for i in range(3)]
        self.bell_proc = env.process(self.bell())

    def bell(self):
        for i in range(2):
            yield self.env.timeout(45)
            self.class_ends.succeed()
            self.class_ends = self.env.event()
            print(self.env.now)

    def pupil(self):
        for i in range(2):
            print(' \o/', end='')
            yield self.class_ends

school = School(env)
env.run()

 \o/ \o/ \o/225
 \o/ \o/ \o/270


In [23]:
from simpy.util import start_delayed

def sub(env):
    yield env.timeout(1)
    return 23

def parent(env):
    start = env.now
    sub_proc = yield start_delayed(env, sub(env), delay=3)
    assert env.now - start == 3
    ret = yield sub_proc
    return ret

env.run(env.process(parent(env)))

23

In [24]:
from simpy.events import AnyOf, AllOf, Event
events = [Event(env) for i in range(3)]
a = AnyOf(env, events)  # Triggers if at least one of "events" is triggered.
b = AllOf(env, events)  # Triggers if all each of "events" is triggered.

In [27]:
import simpy

def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)
    
env = simpy.Environment()
car = env.process(car(env))
env.run(until=40)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
Start driving at 19
Start parking at 21
Start driving at 26
Start parking at 28
Start driving at 33
Start parking at 35


In [30]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)

    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))
        
env = simpy.Environment()
bcs = simpy.Resource(env,capacity = 2)
for i in range(4):
    env.process(car(env,"car:"+repr(i),bcs,10,5))
env.run(until=40)

car:0 arriving at 10
car:1 arriving at 10
car:2 arriving at 10
car:3 arriving at 10
car:0 starting to charge at 10
car:1 starting to charge at 10
car:0 leaving the bcs at 15
car:1 leaving the bcs at 15
car:2 starting to charge at 15
car:3 starting to charge at 15
car:2 leaving the bcs at 20
car:3 leaving the bcs at 20


In [ ]:
import random

import simpy

